## Python Implementation

In [194]:
#This command imports the Gurobi functions and classes.
import gurobipy as gp
from gurobipy import GRB

import pandas as pd
from pylab import *
import matplotlib
import matplotlib.pyplot as plt

## Input Data

We define all the input data of the model.


In [195]:
# toal number of shifts required for each day
day, shiftRequirements = gp.multidict({
  "Mon": 4,
  "Tue": 4,
  "Wed": 5,
  "Thu": 6,
  "Fri": 6,
  "Sat": 8,
  "Sun": 7})

In [196]:
# Worker availability: defines on which day each nail technician worker is available.
availability = gp.tuplelist([
('Penny', 'Mon'), ('Penny', 'Tue'), ('Penny', 'Thur'), ('Penny', 'Sat'),('Penny', 'Sun'),
('Billy', 'Mon'), ('Billy', 'Tue'), ('Billy', 'Wed'), ('Billy', 'Thur'),
('Billy', 'Friday'), ('Alex', 'Tue'), ('Alex', 'Wed'), ('Alex', 'Friday'),
('Alex', 'Sat'), ('Alex', 'Sun'), ('Blenda', 'Mon'), ('Blenda', 'Tue'),
('Blenda', 'Thur'), ('Blenda', 'Fri'), ('Blenda', 'Sat'), ('Ruiyuan', 'Mon'),
('Ruiyuan', 'Wed'), ('Ruiyuan', 'Thur'), ('Ruiyuan', 'Fri'), ('Ruiyuan', 'Sun'),
])

In [197]:
workers, profits = gp.multidict({
  'Penny': 100,
  'Billy': 90,
  'Alex':85,
  'Blenda':110,
  'Ruiyuan':90,
})

noviceprofit = 30

## Model Deployment

In [198]:
# Create initial model.
m = gp.Model("nail")

### Decision Variables


In [199]:
#variable - total shifts each technician assigned to a day
technician = m.addVars(availability, vtype=GRB.BINARY, name="technician")

In [200]:
#variable - total shifts of novice assigned to each day
novice = m.addVars(day, vtype=GRB.INTEGER, name="novice")

In [201]:
#total shifts each technician assigned to next week
totShifts = m.addVars(workers) #, name="TotShifts"

In [202]:
#totnovice = m.addVar(name='totnovice')

In [203]:
#total profits the nail shop earned next week
totprofits = m.addVar(name='totprofits')

### Constraint

In [204]:
m.addConstrs((technician.sum('*',s) + novice[s] == shiftRequirements[s] for s in day), name='shiftRequirement')

{'Mon': <gurobi.Constr *Awaiting Model Update*>,
 'Tue': <gurobi.Constr *Awaiting Model Update*>,
 'Wed': <gurobi.Constr *Awaiting Model Update*>,
 'Thu': <gurobi.Constr *Awaiting Model Update*>,
 'Fri': <gurobi.Constr *Awaiting Model Update*>,
 'Sat': <gurobi.Constr *Awaiting Model Update*>,
 'Sun': <gurobi.Constr *Awaiting Model Update*>}

In [205]:
m.addConstrs((totShifts[w] == technician.sum(w,'*') for w in workers), name='totShifts')

{'Penny': <gurobi.Constr *Awaiting Model Update*>,
 'Billy': <gurobi.Constr *Awaiting Model Update*>,
 'Alex': <gurobi.Constr *Awaiting Model Update*>,
 'Blenda': <gurobi.Constr *Awaiting Model Update*>,
 'Ruiyuan': <gurobi.Constr *Awaiting Model Update*>}

### Constraint

In [206]:
m.addConstr(totprofits == (sum(profits[i]*totShifts[i] for i in workers) + (novice.sum())*noviceprofit), name='profits')

<ipython-input-206-2b21a02694a7>:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  m.addConstr(totprofits == (sum(profits[i]*totShifts[i] for i in workers) + (novice.sum())*noviceprofit), name='profits')


<gurobi.Constr *Awaiting Model Update*>

In [207]:
minShift = m.addVar(name='minShift')

maxShift = m.addVar(name='maxShift')

min_constr = m.addGenConstrMin(minShift, totShifts, name='minShift')

max_constr = m.addGenConstrMax(maxShift, totShifts, name='maxShift')

## Objective Function

In [208]:
m.ModelSense = GRB.MINIMIZE

In [209]:
m.setObjectiveN(maxShift - minShift, index=1, priority=1, name='Fairness')

In [210]:
m.setObjectiveN((-totprofits), index=1, priority=2, name='profit')

In [211]:
m.write('nail.lp')

In [212]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 40 columns and 69 nonzeros
Model fingerprint: 0x52a81bae
Model has 2 general constraints
Variable types: 8 continuous, 32 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 8e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 4 rows and 12 columns
Presolve time: 0.01s
Presolved: 17 rows and 52 columns
---------------------------------------------------------------------------

Multi-objectives: op

In [213]:
status = m.Status
if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE  or status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(0)
if status != GRB.Status.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(0)

In [214]:
for x in m.getVars():
    print(x.varName, x.x)

technician[Penny,Mon] 1.0
technician[Penny,Tue] 1.0
technician[Penny,Thur] 1.0
technician[Penny,Sat] 1.0
technician[Penny,Sun] 1.0
technician[Billy,Mon] 1.0
technician[Billy,Tue] 1.0
technician[Billy,Wed] 1.0
technician[Billy,Thur] 1.0
technician[Billy,Friday] 1.0
technician[Alex,Tue] 1.0
technician[Alex,Wed] 1.0
technician[Alex,Friday] 1.0
technician[Alex,Sat] 1.0
technician[Alex,Sun] 1.0
technician[Blenda,Mon] 1.0
technician[Blenda,Tue] 1.0
technician[Blenda,Thur] 1.0
technician[Blenda,Fri] 1.0
technician[Blenda,Sat] 1.0
technician[Ruiyuan,Mon] 1.0
technician[Ruiyuan,Wed] 1.0
technician[Ruiyuan,Thur] 1.0
technician[Ruiyuan,Fri] 1.0
technician[Ruiyuan,Sun] 1.0
novice[Mon] 0.0
novice[Tue] 0.0
novice[Wed] 2.0
novice[Thu] 6.0
novice[Fri] 4.0
novice[Sat] 5.0
novice[Sun] 4.0
C32 5.0
C33 5.0
C34 5.0
C35 5.0
C36 5.0
totprofits 3005.0
minShift 5.0
maxShift 5.000000000000001
